In [5]:
file_path_ = ""

In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn

from sklearn.model_selection import train_test_split

In [7]:
data_csv = pd.read_csv( file_path_ + "data.csv" , dtype={'label': 'int32' })

In [9]:
data_csv.head(1)

,f1,f2,label
0,0.450564,1.074305,0


In [21]:
tf.__version__  , tf.executing_eagerly()

('2.3.0', True)

In [ ]:
# data_csv.drop( ['label'], axis=1)

In [11]:
# data_csv
data_y = data_csv['label'].values
data_x = data_csv.drop( ['label'], axis=1).values
data_x.shape , data_y.shape 

((20000, 2), (20000,))

In [12]:

#data_y = data_csv['label'].values
#data_x = data_csv.drop( ['label'], axis=1).values
# split the data into test and train by maintaining same distribution of output varaible 'data_y' [stratify=data_y]
X_train, X_test, Y_train, Y_test = train_test_split( data_x , data_y,stratify=data_y,test_size=0.20)


In [14]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((16000, 2), (16000,), (4000, 2), (4000,))

In [ ]:
class LossHistory(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        self.history={'loss': [],'acc': [],'val_loss': [],'val_acc': []}
        
    def on_epoch_end(self, epoch, logs={}):
        ## on end of each epoch, we will get logs and update the self.history dict
        self.history['loss'].append(logs.get('loss'))
        self.history['acc'].append(logs.get('acc'))
        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_acc', -1) != -1:
            self.history['val_acc'].append(logs.get('val_acc'))

In [ ]:
class LearningRateCustom(tf.keras.callbacks.Callback):
      def __init__(self, schedule, verbose=0):
        super(LearningRateCustom, self).__init__()
        self.schedule = schedule
        self.verbose = verbose

      def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
          raise ValueError('Optimizer must have a "lr" attribute.')
        try:  # new API
          lr = float(K.get_value(self.model.optimizer.lr))
          lr = self.schedule(epoch, lr)
        except TypeError:  # Support for old API for backward compatibility
          lr = self.schedule(epoch)
        if not isinstance(lr, (ops.Tensor, float, np.float32, np.float64)):
          raise ValueError('The output of the "schedule" function '
                          'should be float.')
        if isinstance(lr, ops.Tensor) and not lr.dtype.is_floating:
          raise ValueError('The dtype of Tensor should be float')
        K.set_value(self.model.optimizer.lr, K.get_value(lr))
        if self.verbose > 0:
          print('\nEpoch %05d: LearningRateScheduler reducing learning '
                'rate to %s.' % (epoch + 1, lr))

      def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [ ]:
#Custom Callback class
class LearningRate(tf.keras.callbacks.LearningRateScheduler):
    def __init__(self ,  schedule, verbose=0 ):
        print("__init__ called")
        super(LearningRate, self ).__init__(  schedule, verbose )
        self.schedule = schedule
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        print("on LearningRate called")
       # self.history_data={'loss': [],'acc': [],'val_loss': [],'val_acc': []}
        self.initial_learningrate = 0.1
        
    def on_epoch_end(self, epoch, logs=None):
        ## on end of each epoch, we will get logs and update the self.history dict
        print("on_epoch_end called")
        print("logs : " , logs )
        self.schedule(epoch)
        '''self.history_data['loss'].append(logs.get('loss'))
        self.history_data['acc'].append(logs.get('accuracy'))
        if logs.get('val_loss', -1) != -1:
            self.history_data['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_accuracy', -1) != -1:
            self.history_data['val_acc'].append(logs.get('val_accuracy'))

        # getting any NaN values(either weigths or loss) while training, it has to terminate your training.
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print("Invalid loss and terminated at epoch {}".format(epoch))
                self.model.stop_training = True
        '''


In [25]:
#Custom Callback class
class LossHistoryData(tf.keras.callbacks.Callback):
    def __init__(self):
        print("__init__ LossHistoryData")

    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history_data
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        #print("on_train_begin called")
        self.history_data={'loss': [],'accuracy': [],'val_loss': [],'val_accuracy': []}
        
    def on_epoch_end(self, epoch, logs={}):
        ## on end of each epoch, we will get logs and update the self.history_data dict
        #print("on_epoch_end called")
        print("epoch:" , epoch)
        self.history_data['loss'].append(logs.get('loss'))
        self.history_data['accuracy'].append(logs.get('accuracy'))
        if logs.get('val_loss', -1) != -1:
            self.history_data['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_accuracy', -1) != -1:
            self.history_data['val_accuracy'].append(logs.get('val_accuracy'))

        if (epoch > 2):
          val_acc = self.history_data.get("val_accuracy")
          if ( ( val_acc[epoch-1] < val_acc[epoch-2] ) and  ( val_acc[epoch-2] < val_acc[epoch-3]) ):
            #print("val_acc is not increased in last 2 epoch{}".format(val_acc[epoch-3 : epoch]))
            print("val_acc is not increased in last 2 epoch")
            self.model.stop_training = True
        # getting any NaN values(either weigths or loss) while training, it has to terminate your training.
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print("Invalid loss and terminated at epoch {}".format(epoch))
                self.model.stop_training = True

In [16]:
#initial_learningrate=0.1
def changeLearningRate(epoch , lr ):
        #print( "---> changeLearningRate : " , epoch )
        #initial_learningrate=0.01
        val_loss = history_own_data.history_data.get("val_accuracy")
        #print(lr , "--->v_acc" , val_loss)
        #lr = ( lr ) *(0.1)
        if ( epoch > 1):
          if ( (epoch % 3) == 0):
            lr = ( lr ) *(0.05)
          elif (val_loss[epoch-1] < val_loss[epoch-2]):
            lr = ( lr ) *(0.1)
        
        return lr

In [18]:
#changeLearningRate(None, None)

In [19]:
Y_train = tf.keras.utils.to_categorical(Y_train, 10) 
Y_test = tf.keras.utils.to_categorical(Y_test, 10)

In [92]:
#from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc , roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint

In [80]:
class F1ScoreMetrics(tf.keras.callbacks.Callback):
    def __init__(self , validation ):   
        super(F1ScoreMetrics, self).__init__()
        self.validation_data = validation    
            
        print('validation shape', len(self.validation_data[0]))
        print("__init__F1ScoreMetrics")

    def on_train_begin(self, logs={}):        
        self.val_f1score = []
        self.auc_score = []
        #self.val_recalls = []
        #self.val_precisions = []
     
    def on_epoch_end(self, epoch, logs={}):
        #val_targ = self.validation[1]   
        #val_predict = (np.asarray(self.model.predict(self.validation[0]))).round()        
        val_x = self.validation_data[0]
        val_y = self.validation_data[1]
        val_predicted = (np.asarray(self.model.predict(val_x))).round() #(np.asarray()).round()
        #print("val_predicted : " , val_predicted.shape )
        #print("val_y : " , val_y.shape )
        val_f1 = f1_score(val_y, val_predicted ,  average='micro' )
        #val_recall = recall_score(val_targ, val_predict)         
        #val_precision = precision_score(val_targ, val_predict)
        #fpr,tpr,ths = roc_curve(np.mean(val_y, axis=1 ), np.mean( val_predicted , axis=1 ) )
        auc_sc = roc_auc_score( val_y, val_predicted , average='micro')
        #auc_sc = auc(fpr, tpr)
        
        self.val_f1score.append(round(val_f1, 6))
        self.auc_score.append( auc_sc )
        #self.val_recalls.append(round(val_recall, 6))
        #self.val_precisions.append(round(val_precision, 6))
 
        #print(f' — val_f1: {val_f1} — val_precision: {val_precision}, — val_recall: {val_recall}')
        print(f' — val_f1score: {val_f1} -- AUC : {auc_sc}' )

In [110]:
import os

if (not os.path.isdir("/home/model_save")):
    print(os.makedirs("/home/model_save"))

In [111]:
#to work with callback

#Input layer
input_layer = Input(shape=(2,))

#initializer = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
#Dense hidden layer
layer1 = Dense(50,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(input_layer)

layer2 = Dense(40,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer1)
layer3 = Dense(30,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer2)
layer4 = Dense(20,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer3)
layer5 = Dense(10,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer4)

#output layer
output = Dense(10,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

optimizer = tf.keras.optimizers.Adam(0.01)
#opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
#Callbacks
history_own_data = LossHistoryData()
#history_own_data = LossHistory()
lrschedule = tf.keras.callbacks.LearningRateScheduler( changeLearningRate, verbose=1)
#lrschedule = LearningRate(changeLearningRate , verbose=1)
f1score = F1ScoreMetrics( validation=(X_test,Y_test))
filepath="/home/model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')

call_back_list = [history_own_data , lrschedule , f1score , checkpoint ]
#optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,Y_train,epochs=3, validation_data=(X_test,Y_test), batch_size=16, callbacks= call_back_list )


__init__ LossHistoryData
validation shape 4000
__init__F1ScoreMetrics

Epoch 00001: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 1/3
 987/1000 [============================>.] - ETA: 0s - loss: 0.7676 - accuracy: 0.4987epoch: 0
 — val_f1score: 0.5 -- AUC : 0.7222222222222221

Epoch 00001: val_loss improved from inf to 0.69575, saving model to /home/model_save/weights-01-0.5000.hdf5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.7666 - accuracy: 0.4991 - val_loss: 0.6958 - val_accuracy: 0.5000

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 2/3
 993/1000 [============================>.] - ETA: 0s - loss: 0.6953 - accuracy: 0.5023epoch: 1
 — val_f1score: 0.5 -- AUC : 0.7222222222222221

Epoch 00002: val_loss improved from 0.69575 to 0.69451, saving model to /home/model_save/weights-02-0.5000.hdf5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6954 - accuracy: 0.5019 - val_loss

In [113]:
model.load_weights('/home/model_save/weights-02-0.5000.hdf5')
model.fit(X_train,Y_train)

500/500 [==============================] - 1s 2ms/step - loss: 0.6942 - accuracy: 0.4945


In [ ]:
history_own_data.history_data.get("val_accuracy")[0:4]

[0.5024999976158142,
 0.49300000071525574,
 0.47350001335144043,
 0.4737499952316284]

In [ ]:
history_own_data.history_data

{'acc': [0.5019375085830688, 0.5208125114440918, 0.6370000243186951],
 'loss': [0.7112691402435303, 0.6974461674690247, 0.6364936232566833],
 'val_acc': [0.5, 0.5137500166893005, 0.6667500138282776],
 'val_loss': [0.69703209400177, 0.6818374395370483, 0.6041727662086487]}

In [ ]:
history_own_data.history

{'acc': [None, None, None],
 'loss': [0.7149479985237122, 0.6955052018165588, 0.6351665258407593],
 'val_acc': [],
 'val_loss': [0.6971844434738159, 0.6879738569259644, 0.6043661236763]}

In [114]:
#to work with callback

#Input layer
input_layer1 = Input(shape=(2,))

initializer = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
#Dense hidden layer
layer1_1 = Dense(50,activation='tanh',kernel_initializer=initializer)(input_layer1)
layer1_2 = Dense(40,activation='tanh',kernel_initializer=initializer)(layer1_1)
layer1_3 = Dense(30,activation='tanh',kernel_initializer=initializer)(layer1_2)
layer1_4 = Dense(20,activation='tanh',kernel_initializer=initializer)(layer1_3)
layer1_5 = Dense(10,activation='tanh',kernel_initializer=initializer)(layer1_4)
#output layer
output1 = Dense(10,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer1_5)
#Creating a model
model1 = Model(inputs=input_layer1,outputs=output1)

#optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
#Callbacks
history_own_data1 = LossHistoryData()
lrschedule = tf.keras.callbacks.LearningRateScheduler( changeLearningRate, verbose=1)
f1score = F1ScoreMetrics( validation=(X_test,Y_test))
filepath="/home/model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')

call_back_list = [history_own_data1 , lrschedule , f1score , checkpoint]

model1.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model1.fit(X_train,Y_train,epochs=3, validation_data=(X_test,Y_test), batch_size=16, callbacks= call_back_list )


__init__ LossHistoryData
validation shape 4000
__init__F1ScoreMetrics

Epoch 00001: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 1/3
 983/1000 [============================>.] - ETA: 0s - loss: 1.2280 - accuracy: 0.4671epoch: 0
 — val_f1score: 0.6666666666666666 -- AUC : 0.9444444444444444

Epoch 00001: val_loss improved from inf to 0.83023, saving model to /home/model_save/weights-01-0.5070.hdf5
1000/1000 [==============================] - 2s 2ms/step - loss: 1.2212 - accuracy: 0.4678 - val_loss: 0.8302 - val_accuracy: 0.5070

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 2/3
 955/1000 [===========================>..] - ETA: 0s - loss: 0.7777 - accuracy: 0.5086epoch: 1
 — val_f1score: 0.6666666666666666 -- AUC : 0.9444444444444444

Epoch 00002: val_loss improved from 0.83023 to 0.74643, saving model to /home/model_save/weights-02-0.5120.hdf5
1000/1000 [==============================] - 1s 1ms/step - loss: 0.7764

In [115]:
#to work with callback

#Input layer
input_layer2 = Input(shape=(2,))

initializer = tf.keras.initializers.he_uniform()
#Dense hidden layer
layer2_1 = Dense(50,activation='relu',kernel_initializer=initializer)(input_layer2)
layer2_2 = Dense(40,activation='relu',kernel_initializer=initializer)(layer2_1)
layer2_3 = Dense(30,activation='relu',kernel_initializer=initializer)(layer2_2)
layer2_4 = Dense(20,activation='relu',kernel_initializer=initializer)(layer2_3)
layer2_5 = Dense(10,activation='relu',kernel_initializer=initializer)(layer2_4)
#output layer
output2 = Dense(10,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer2_5)
#Creating a model
model2 = Model(inputs=input_layer2,outputs=output2)

#optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
#Callbacks
history_own_data2 = LossHistoryData()
lrschedule = tf.keras.callbacks.LearningRateScheduler( changeLearningRate, verbose=1)
f1score = F1ScoreMetrics( validation=(X_test,Y_test))
filepath="/home/model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')

call_back_list = [history_own_data2 , lrschedule , f1score , checkpoint ]

model2.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model2.fit(X_train,Y_train,epochs=3, validation_data=(X_test,Y_test), batch_size=16, callbacks= call_back_list )


__init__ LossHistoryData
validation shape 4000
__init__F1ScoreMetrics

Epoch 00001: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 1/3
 957/1000 [===========================>..] - ETA: 0s - loss: 0.7129 - accuracy: 0.5895epoch: 0
 — val_f1score: 0.6666666666666666 -- AUC : 0.9444444444444444

Epoch 00001: val_loss improved from inf to 0.69320, saving model to /home/model_save/weights-01-0.5850.hdf5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.7121 - accuracy: 0.5897 - val_loss: 0.6932 - val_accuracy: 0.5850

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 2/3
 977/1000 [============================>.] - ETA: 0s - loss: 0.6931 - accuracy: 0.5621epoch: 1
 — val_f1score: 0.6666666666666666 -- AUC : 0.9444444444444444

Epoch 00002: val_loss improved from 0.69320 to 0.68993, saving model to /home/model_save/weights-02-0.5000.hdf5
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6930

In [116]:
#to work with callback

#Input layer
input_layer3 = Input(shape=(2,))

initializer = tf.keras.initializers.he_uniform()
#Dense hidden layer
layer3_1 = Dense(50,activation='relu',kernel_initializer=initializer)(input_layer3)
layer3_2 = Dense(40,activation='relu',kernel_initializer=initializer)(layer3_1)
layer3_3 = Dense(30,activation='relu',kernel_initializer=initializer)(layer3_2)
layer3_4 = Dense(20,activation='relu',kernel_initializer=initializer)(layer3_3)
layer3_5 = Dense(10,activation='relu',kernel_initializer=initializer)(layer3_4)
#output layer
output3 = Dense(10,activation='sigmoid',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer3_5)
#Creating a model
model3 = Model(inputs=input_layer3,outputs=output3)

#optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
#Callbacks
history_own_data3 = LossHistoryData()
lrschedule = tf.keras.callbacks.LearningRateScheduler( changeLearningRate, verbose=1)
f1score = F1ScoreMetrics( validation=(X_test,Y_test))
filepath="/home/model_save/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss',  verbose=1, save_best_only=True, mode='auto')

call_back_list = [history_own_data3 , lrschedule , f1score , checkpoint ]

model3.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model3.fit(X_train,Y_train,epochs=3, validation_data=(X_test,Y_test), batch_size=16, callbacks= call_back_list )


__init__ LossHistoryData
validation shape 4000
__init__F1ScoreMetrics

Epoch 00001: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 1/3
 993/1000 [============================>.] - ETA: 0s - loss: 0.7246 - accuracy: 0.6053epoch: 0
 — val_f1score: 0.6666666666666666 -- AUC : 0.9444444444444444

Epoch 00001: val_loss improved from inf to 0.69320, saving model to /home/model_save/weights-01-0.6135.hdf5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.7244 - accuracy: 0.6055 - val_loss: 0.6932 - val_accuracy: 0.6135

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009999999776482582.
Epoch 2/3
 993/1000 [============================>.] - ETA: 0s - loss: 0.6932 - accuracy: 0.6209epoch: 1
 — val_f1score: 0.6666666666666666 -- AUC : 0.9444444444444444

Epoch 00002: val_loss improved from 0.69320 to 0.69317, saving model to /home/model_save/weights-02-0.6160.hdf5
1000/1000 [==============================] - 1s 1ms/step - loss: 0.6932